# Load Louvain 3

In [1]:
import pandas as pd
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [15]:
from copy import deepcopy

In [7]:
df = {}

In [12]:
df['lv3-ini'] = pd.read_csv('../data/MMC10_Li3_Tcell_liveCD45/Louvain_3/summary.csv', index_col=0)

In [13]:
# net.load_file('../data/MMC10_Li3_Tcell_liveCD45/Louvain_3/summary.csv')
# df['lv3'] = net.export_df()

In [14]:
df['lv3-ini'].head()

,Louvain_3,Treatment,N,Total,Freq,CD45,CD57,In115Di,IdU,CD103,...,CD38,CD95,LAG3,CXCR3,HLADR,TIGIT,GranzymeB,DNA,Osmium,CD11b
1,6,1_normal,8230,169733,0.048488,0.658259,0.169114,0.033096,0.240033,0.049980,...,0.142250,0.425633,0.091767,0.183889,0.122041,0.055854,0.435562,0.804734,0.862120,0.734667
2,6,2_border,250,55703,0.004488,0.690238,0.035653,0.012406,0.149922,0.040909,...,0.114886,0.371667,0.069548,0.203976,0.140674,0.046681,0.385222,0.814625,0.840464,0.764056
3,6,3_tumor,235,333723,0.000704,0.672674,0.034560,0.009759,0.216150,0.062991,...,0.158168,0.378995,0.079664,0.255813,0.148387,0.069659,0.409060,0.808674,0.823157,0.707447
4,7,1_normal,21617,169733,0.127359,0.429158,0.018248,0.005821,0.130442,0.052624,...,0.115924,0.278340,0.044500,0.088673,0.058971,0.042087,0.238030,0.751865,0.406691,0.569863
5,7,2_border,1015,55703,0.018222,0.531833,0.020442,0.006815,0.115594,0.047920,...,0.123638,0.322280,0.052708,0.097743,0.067522,0.038370,0.227413,0.767207,0.434562,0.678852


In [18]:
cols = df['lv3-ini'].columns.tolist()

In [22]:
markers = cols[5:]
print(markers)

['CD45', 'CD57', 'In115Di', 'IdU', 'CD103', 'CD19', 'CD45RA', 'KLRG1', 'CD4', 'CD8', 'Perforin', 'CD16', 'CD127', 'CD1c', 'CD123', 'CD66b', 'PD1', 'ICOS', 'CD27', 'TIM3', 'CCR6', 'CD14', 'CD56', 'CD69', '41BB', 'CD25', 'CCR7', 'CD3', 'OX40', 'CD38', 'CD95', 'LAG3', 'CXCR3', 'HLADR', 'TIGIT', 'GranzymeB', 'DNA', 'Osmium', 'CD11b']


In [38]:
cluster_cats = {}
for inst_col in cols[:5]:

    inst_cats = df['lv3-ini'][inst_col].get_values().tolist()
    
    cluster_cats[inst_col] = inst_cats
    

In [43]:
rows = df['lv3-ini'].index.tolist()
print(len(rows))

87


In [72]:
for inst_row in range(len(rows[:4])):
    inst_tuple = ()
    print('\n')

    # gather category info for inst_row
    inst_dict = {}    

    inst_cats = cols[:5]
    
    for inst_cat_title in cols[:5]:

        if inst_cat_title != 'N' and inst_cat_title != 'Total':
            
            # inst_row is the index
            inst_cat = str(cluster_cats[inst_cat_title][inst_row])
            
            # add 'Cluster' to cluster index
            if 'Louvain_' in inst_cat_title:
                inst_cat = 'Cluster-' + inst_cat
            
            # remove number from 'treatment'
            if 'Treatment' in inst_cat_title:
                inst_cat = inst_cat.split('_')[1]
            
            inst_cat_name = inst_cat_title + ': ' + inst_cat

            # add to inst_dict
            inst_dict[inst_cat_title] = inst_cat_name
    
    # manually generate name
    inst_name = inst_dict[inst_cats[0]].split(': ')[1] + '_' + inst_dict[inst_cats[1]].split(': ')[1]
    
    print(inst_name)
    
#             inst_tuple = inst_tuple + (inst_col, )



Cluster-6_normal


Cluster-6_border


Cluster-6_tumor


Cluster-7_normal


In [56]:
inst_tuple

('Louvain_3', 'Treatment', 'Freq')

In [29]:
LV3_index = df['lv3-ini']['Louvain_3'].get_values().tolist()

In [ ]:
sample_location = df['lv3-ini']['']

In [30]:
# net.load_df(df['lv3'])
# net.cluster()
# net.widget()